In [2]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Всего 4361 женщина.
Хотим научиться оценивать количество детей ceb по остальным признакам.
О каждой из них мы знаем:

сколько детей она родила (признак ceb)
возраст (age)
длительность получения образования (educ)
религиозная принадлежность (religion)
идеальное, по её мнению, количество детей в семье (idlnchld)
была ли она когда-нибудь замужем (evermarr)
возраст первого замужества (agefm)
длительность получения образования мужем (heduc)
знает ли она о методах контрацепции (knowmeth)
использует ли она методы контрацепции (usemeth)
живёт ли она в городе (urban)
есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

In [4]:
data = pd.read_csv('botswana.tsv', sep = '\t')

In [5]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [6]:
data.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

In [7]:
data.columns

Index([u'ceb', u'age', u'educ', u'religion', u'idlnchld', u'knowmeth',
       u'usemeth', u'evermarr', u'agefm', u'heduc', u'urban', u'electric',
       u'radio', u'tv', u'bicycle'],
      dtype='object')

In [8]:
new_data = data.dropna()

In [9]:
new_data.shape

(1834, 15)

In [10]:
data.shape

(4361, 15)

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак x_1x 
1
​	  на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

создать новый бинарный признак
x2={1,0,x1='не применимо',иначе;
заменить "не применимо" в x1 на произвольную константу c, которая среди других значений x1 не встречается.
Теперь, когда мы построим регрессию на оба признака и получим модель вида
y=\beta_0 + \beta_1 x_1 + \beta_2 x_2,y=β 
0
​	 +β 
1
​	 x 
1
​	 +β 
2
​	 x 
2
​	 ,
на тех объектах, где x_1x 
1
​	  было измерено, регрессионное уравнение примет вид
y=\beta_0 + \beta_1 x,y=β 
0
​	 +β 
1
​	 x,
а там, где x_1x 
1
​	  было "не применимо", получится
y=\beta_0 + \beta_1 c + \beta_2.y=β 
0
​	 +β 
1
​	 c+β 
2
​	 .
Выбор cc влияет только на значение и интерпретацию \beta_2β 
2
​	 , но не \beta_1β 
1
​	 .

Давайте используем этот метод для обработки пропусков в agefm и heduc.

Создайте признак nevermarr, равный единице там, где в agefm пропуски.
Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.
Замените NaN в признаке agefm на cagefm=0.
У объектов, где nevermarr = 1, замените NaN в признаке heduc на cheduc1=−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).
Сколько осталось пропущенных значений в признаке heduc?

In [11]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [12]:
data['nevermarr'] = data['agefm'].apply(lambda x : 1 if pd.isnull(x) else 0)

In [13]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [14]:
data['nevermarr'].value_counts()

1    2282
0    2079
Name: nevermarr, dtype: int64

In [15]:
data['agefm'].value_counts().sum()

2079

In [16]:
data.drop('evermarr',axis=1, inplace=True)

In [17]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [18]:
data['agefm'] = data['agefm'].fillna(0)

In [19]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [22]:
for i in range(data.shape[0]):
    if data['nevermarr'][i] == 1:
        data['heduc'][i] = -1

C:\Users\Nastya\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [28]:
data['heduc'].isnull().sum()

123

In [29]:
data['idlnchld_noans'] = data['idlnchld'].apply(lambda x: 1 if pd.isnull(x) else 0)
data['heduc_noans']  = data['heduc'].apply(lambda x: 1 if pd.isnull(x) else 0) 
data['usemeth_noans'] = data['usemeth'].apply(lambda x: 1 if pd.isnull(x) else 0)

In [30]:
data['idlnchld'] = data['idlnchld'].fillna(-1)
data['heduc']  = data['heduc'].fillna(-2)
data['usemeth'] = data['usemeth'].fillna(-1)

In [31]:
data.isnull().sum()

ceb               0
age               0
educ              0
religion          0
idlnchld          0
knowmeth          7
usemeth           0
agefm             0
heduc             0
urban             0
electric          3
radio             2
tv                2
bicycle           3
nevermarr         0
idlnchld_noans    0
heduc_noans       0
usemeth_noans     0
dtype: int64

In [34]:
data.dropna(inplace = True)

In [35]:
data.shape

(4348, 18)

In [225]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [36]:
4348*18

78264

In [37]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth +'\
                    'agefm + heduc + urban + electric+radio+tv+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans', 
             data=data)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Tue, 26 Feb 2019   Prob (F-statistic):               0.00
Time:                        19:03:44   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [39]:
sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1]

1.1452927633437192e-225

In [40]:
m4 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth +'\
                    'agefm + heduc + urban + electric+radio+tv+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans', 
             data=data)
fitted_4 = m4.fit(cov_type='HC1')
print fitted_4.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Tue, 26 Feb 2019   Prob (F-statistic):               0.00
Time:                        19:04:33   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [41]:
data_deleted = data.drop('radio', axis = 1)
data_deleted = data_deleted.drop('tv', axis = 1)
data_deleted = data_deleted.drop('religion', axis = 1)

In [42]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth +'\
                    'agefm + heduc + urban + electric+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans', 
             data=data_deleted)
fitted_deleted = m2.fit()
print fitted_deleted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Tue, 26 Feb 2019   Prob (F-statistic):               0.00
Time:                        19:05:31   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [45]:
sms.het_breuschpagan(fitted_deleted.resid, fitted_deleted.model.exog)[1]

1.1197458896531511e-228

In [47]:
m2_1 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth +'\
                    'agefm + heduc + urban + electric+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans', 
             data=data_deleted)
fitted_deleted_1 = m2.fit(cov_type='HC1')
print fitted_deleted_1.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Tue, 26 Feb 2019   Prob (F-statistic):               0.00
Time:                        19:07:50   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [52]:
print "F=%f, p=%f, k1=%f" % m4.fit().compare_f_test(m2_1.fit())

F=0.919236, p=0.467231, k1=5.000000


In [53]:
data_deleted_2 = data_deleted.drop('usemeth', axis = 1)
data_deleted_2 = data_deleted_2.drop('usemeth_noans', axis = 1)

In [54]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth +'\
                    'agefm + heduc + urban + electric+bicycle+nevermarr+idlnchld_noans+heduc_noans', 
             data=data_deleted_2)
fitted_deleted_2 = m3.fit()
print fitted_deleted_2.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     611.3
Date:                Tue, 26 Feb 2019   Prob (F-statistic):               0.00
Time:                        19:09:50   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.202     -5.

In [57]:
print "F=%f, p=%f, k1=%f" % m2_1.fit().compare_f_test(m3.fit())

F=92.890582, p=0.000000, k1=2.000000


In [56]:
m2.fit().compare_f_test(m3.fit())[1]

3.1552009480386492e-40

In [58]:
0.5266 + 0.0705*(-1)

0.45609999999999995